In [1]:
library(httr)
library(jsonlite)

## Prepare the interaction with EdelweissData

In [2]:
# Go to https://edelweissdata.com , log in and go to manage datasets and copy the access token with the button on the top right, then paste it here
access_token <- "" 

In [3]:
base_url <- "https://api.edelweissdata.com"

## Create a dataset in EdelweissData

In [4]:
request_payload <- list(name="Example dataset")

In [5]:
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)
json_payload

{
  "name": "Example dataset"
} 

In [6]:
create_url <- paste(base_url, "datasets", "create", sep = "/")
create_url

[1] "https://api.edelweissdata.com/datasets/create"

In [7]:
res <- POST(create_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)
json <- content(res, "text")
create_reply <- fromJSON(json)
create_reply

$id
[1] "8bedd79e-3ea4-469c-bd8a-c651053e7d6e"

$name
[1] "Example dataset"

$schema
NULL

$created
[1] "2020-09-20T11:07:28.2026278+00:00"

$description
[1] ""

$metadata
named list()

$dataSource
NULL

Now extract the DatasetID from the returned json

In [8]:
dataset_id <- create_reply$id
dataset_id

[1] "8bedd79e-3ea4-469c-bd8a-c651053e7d6e"

## Upload the CSV data file

In [9]:
upload_url <- paste(base_url, "datasets", dataset_id, "in-progress", "data", "upload", sep = "/")
upload_url

[1] "https://api.edelweissdata.com/datasets/8bedd79e-3ea4-469c-bd8a-c651053e7d6e/in-progress/data/upload"

In [10]:
res <- POST(upload_url,
            body = list(data = upload_file("trivial.csv")),
            encode = "multipart",
            add_headers("Authorization" = paste("bearer", access_token))
           )
stop_for_status(res)
json <- content(res, "text")
upload_reply <- fromJSON(json)
upload_reply

$id
[1] "8bedd79e-3ea4-469c-bd8a-c651053e7d6e"

$name
[1] "Example dataset"

$schema
NULL

$created
[1] "2020-09-20T11:07:28.2026270+00:00"

$description
[1] ""

$metadata
named list()

$dataSource
NULL

## Infer the schema

In [11]:
infer_url <- paste(base_url, "datasets", dataset_id, "in-progress", "schema", "infer", sep = "/")

In [12]:
res <- POST(infer_url,                        
            add_headers("Authorization" = paste("bearer", access_token))
           )
stop_for_status(res)

## Set a human readable description

In [13]:
description <- "Example data description. 
"

In [14]:
update_url <- paste(base_url, "datasets", dataset_id, "in-progress", sep = "/")

In [15]:
request_payload <- list(description=description)
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)

In [16]:
res <- POST(update_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)

## Set the visibility

In [17]:
request_payload <- list(isPublic=TRUE)

In [18]:
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)
json_payload

{
  "isPublic": true
} 

In [19]:
visibility_url <- paste(base_url, "datasets", dataset_id, "permissions", "visibility", sep = "/")
visibility_url

[1] "https://api.edelweissdata.com/datasets/8bedd79e-3ea4-469c-bd8a-c651053e7d6e/permissions/visibility"

In [20]:
res <- POST(visibility_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)

## Publish the dataset

In [21]:
publish_url <- paste(base_url, "datasets", dataset_id, "in-progress", "publish", sep = "/")
request_payload <- list(changelog="Initial version")
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)

In [22]:
res <- POST(publish_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)